<a href="https://colab.research.google.com/github/daniel27a/missp/blob/main/pulp/Resistor_bridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook zawiera rozwiązanie problemu liniowego przy pomocy narzędzia Pulp.

Poniższa komórka tekstu zawiera instrukcję instalującą dodatek Pulp do Notebooka

In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

In [ ]:
"""
Minimalizacja mocy rozproszonej - mostek rezystorowy

https://www.youtube.com/watch?v=yZuVfekIXJ0

"""

import numpy as np
from pulp import *

prob = LpProblem("Minimalizacja mocy rozproszonej",LpMinimize)

# Variables (name, Lo, Hi, Type)

i1=LpVariable("I1", 0.003, 0.005, LpContinuous)
i2=LpVariable("I2", 0.001, 0.003, LpContinuous)
i3=LpVariable("I3", 0.001, 0.003, LpContinuous)
i4=LpVariable("I4", 0.001, 0.003, LpContinuous)
i5=LpVariable("I5", 0.003, 0.005, LpContinuous)
 

# Constants
U = (6.0, 10.0, 4.0, 7.0, 3.0)


# Main problem
prob += U[0]*i1 + U[1]*i2 + U[2]*i3 + U[3]*i4 + U[4]*i5, "Total Power"


# Constraints
prob += i1-i3 == i4, "KI1" 
prob += i2+i3 == i5, "KI2"

prob.writeLP("ResistorBridge.lp")

prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print()

print("Straty:", value(prob.objective), "W\n")  

print("-"*30,"\n")

for v, k in zip(U, prob.variables()):
  print("R",U.index(v)+1,"=",(v/k.varValue/1000),"k ohm") 